In [12]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

In [13]:
df = pd.read_csv("powergrid_realistic_material_dataset_with_dates.csv")

# Convert to datetime
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])



In [14]:
df["project_duration_days"] = (df["end_date"] - df["start_date"]).dt.days
df["start_month"] = df["start_date"].dt.month
df["end_month"] = df["end_date"].dt.month
df["start_year"] = df["start_date"].dt.year
df["end_year"] = df["end_date"].dt.year

# Drop raw date columns
df = df.drop(["start_date", "end_date"], axis=1)

In [15]:
target_cols = [col for col in df.columns if "qty" in col.lower() or "quantity" in col.lower() or "demand" in col.lower()]
if not target_cols:
    raise ValueError("No target columns found — please check dataset column names.")

print(f"Detected Target Columns: {target_cols}")

Detected Target Columns: ['quantity_steel_tons', 'quantity_copper_tons', 'quantity_cement_tons', 'quantity_aluminum_tons', 'quantity_insulators_count', 'quantity_conductors_tons', 'quantity_transformers_count', 'quantity_switchgears_count', 'quantity_cables_count', 'quantity_protective_relays_count', 'quantity_oil_tons', 'quantity_foundation_concrete_tons', 'quantity_bolts_count']


In [16]:
label_encoders = {}
cat_cols = df.select_dtypes(include=["object"]).columns

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [17]:
X = df.drop(columns=target_cols)
Y = df[target_cols]

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [19]:
base_model = xgb.XGBRegressor(
    n_estimators=400,
    learning_rate=0.08,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42
)

multi_model = MultiOutputRegressor(base_model)
multi_model.fit(X_train, Y_train)


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.08, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=7,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=400, n_jobs=None,
                                            num_parallel_tree=None, ...))

In [20]:
Y_pred = pd.DataFrame(multi_model.predict(X_test), columns=target_cols)

metrics = {}
for col in target_cols:
    rmse = np.sqrt(mean_squared_error(Y_test[col], Y_pred[col]))
    mae = mean_absolute_error(Y_test[col], Y_pred[col])
    r2 = r2_score(Y_test[col], Y_pred[col])
    metrics[col] = {"RMSE": rmse, "MAE": mae, "R2": r2}
    print(f"✅ {col} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.3f}")

✅ quantity_steel_tons - RMSE: 4.18, MAE: 2.81, R²: 0.975
✅ quantity_copper_tons - RMSE: 0.28, MAE: 0.19, R²: 0.983
✅ quantity_cement_tons - RMSE: 2.07, MAE: 1.40, R²: 0.984
✅ quantity_aluminum_tons - RMSE: 0.23, MAE: 0.15, R²: 0.982
✅ quantity_insulators_count - RMSE: 28.00, MAE: 19.83, R²: 0.898
✅ quantity_conductors_tons - RMSE: 2.25, MAE: 1.51, R²: 0.983
✅ quantity_transformers_count - RMSE: 0.53, MAE: 0.49, R²: 0.791
✅ quantity_switchgears_count - RMSE: 0.54, MAE: 0.50, R²: 0.971
✅ quantity_cables_count - RMSE: 0.91, MAE: 0.78, R²: 0.972
✅ quantity_protective_relays_count - RMSE: 0.55, MAE: 0.50, R²: 0.984
✅ quantity_oil_tons - RMSE: 0.16, MAE: 0.14, R²: 0.994
✅ quantity_foundation_concrete_tons - RMSE: 1.37, MAE: 0.93, R²: 0.984
✅ quantity_bolts_count - RMSE: 1.44, MAE: 0.94, R²: 1.000


In [ ]:
import joblib
model_bundle = {
    "model": multi_model,
    "label_encoders": label_encoders,
    "columns": list(X.columns),
    "target_cols": target_cols
}

joblib.dump(model_bundle, "multi_material_demand_model.joblib")

print("\n✅ Multi-target model saved as 'multi_material_demand_model.joblib'")


✅ Multi-target model saved as 'multi_material_demand_model.pkl'
